In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'
from optimization import *
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
color_palette = ['#332288',  '#117733', '#44AA99', '#88CCEE', '#DDCC77', '#CC6677', '#AA4499', '#882255']

## Figure 3: Annual V2G Costs 

In [ ]:
def replace_nan_with_previous_value(cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere):
    '''Replace any nan weeks with the previous week's value and reshape the matrices to Jan-Jan format.'''
    for vin in np.arange(cost_mat_controlled.shape[0]):
        for week in np.arange(cost_mat_controlled.shape[1]):
            if np.isnan(cost_mat_controlled[vin, week]):
                if week == 0:
                    cost_mat_controlled[vin, week] = 0
                else:
                    cost_mat_controlled[vin, week] = cost_mat_controlled[vin, week-1]
            if np.isnan(cost_mat_uncontrolled[vin, week]):
                if week == 0:
                    cost_mat_uncontrolled[vin, week] = 0
                else:
                    cost_mat_uncontrolled[vin, week] = cost_mat_uncontrolled[vin, week-1]
            if np.isnan(cost_mat_v2g_home[vin, week]):
                if week == 0:
                    cost_mat_v2g_home[vin, week] = 0
                else:
                    cost_mat_v2g_home[vin, week] = cost_mat_v2g_home[vin, week-1]
            if np.isnan(cost_mat_v2g_everywhere[vin, week]):
                if week == 0:
                    cost_mat_v2g_everywhere[vin, week] = 0
                else:
                    cost_mat_v2g_everywhere[vin, week] = cost_mat_v2g_everywhere[vin, week-1]


    #find rows that sum up to zero from controlled and remove them from cost matrices
    zero_idx = np.all(cost_mat_controlled == 0, axis=1)
    cost_mat_controlled = cost_mat_controlled[~zero_idx]
    cost_mat_uncontrolled = cost_mat_uncontrolled[~zero_idx]
    cost_mat_v2g_home = cost_mat_v2g_home[~zero_idx]
    cost_mat_v2g_everywhere = cost_mat_v2g_everywhere[~zero_idx]

    #order the matrices so data is Jan-Jan
    cost_mat_uncontrolled_jan = np.zeros((cost_mat_uncontrolled.shape[0], 52))
    cost_mat_controlled_jan = np.zeros((cost_mat_controlled.shape[0], 52))
    cost_mat_v2g_home_jan = np.zeros((cost_mat_v2g_home.shape[0], 52))
    cost_mat_v2g_everywhere_jan = np.zeros((cost_mat_v2g_everywhere.shape[0], 52)) 

    num_weeks = 52
    year_splice = 22
    year_splice_2 = num_weeks-year_splice
    cost_mat_uncontrolled_jan[:, 0:year_splice] = cost_mat_uncontrolled[:, year_splice_2:num_weeks]
    cost_mat_uncontrolled_jan[:, year_splice:num_weeks] = cost_mat_uncontrolled[:, 0:year_splice_2]
    cost_mat_controlled_jan[:, 0:year_splice] = cost_mat_controlled[:, year_splice_2:num_weeks] 
    cost_mat_controlled_jan[:, year_splice:num_weeks] = cost_mat_controlled[:, 0:year_splice_2]
    cost_mat_v2g_home_jan[:, 0:year_splice] = cost_mat_v2g_home[:, year_splice_2:num_weeks]
    cost_mat_v2g_home_jan[:, year_splice:num_weeks] = cost_mat_v2g_home[:, 0:year_splice_2]
    cost_mat_v2g_everywhere_jan[:, 0:year_splice] = cost_mat_v2g_everywhere[:, year_splice_2:num_weeks]
    cost_mat_v2g_everywhere_jan[:, year_splice:num_weeks] = cost_mat_v2g_everywhere[:, 0:year_splice_2]

    return cost_mat_uncontrolled_jan, cost_mat_controlled_jan, cost_mat_v2g_home_jan, cost_mat_v2g_everywhere_jan


In [ ]:
def plot_costs(ax, cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere, num_periods, circuit):
    ax.plot(np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0), color = color_palette[0], linewidth = 2, label='Baseline')
    ax.plot(np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0), color = color_palette[1], linewidth = 2, label='Managed')
    ax.plot(np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0), color = color_palette[3], label='V2G Home')
    ax.plot(np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0), color = color_palette[4], label='V2G Everywhere')

    ax.fill_between(np.arange(cost_mat_uncontrolled.shape[1]), np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0) - np.std(np.cumsum(cost_mat_uncontrolled, axis=1), axis=0), np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0) + np.std(np.cumsum(cost_mat_uncontrolled, axis=1), axis=0), color = color_palette[0], alpha = 0.15)
    ax.fill_between(np.arange(cost_mat_controlled.shape[1]), np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0) - np.std(np.cumsum(cost_mat_controlled, axis=1), axis=0), np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0) + np.std(np.cumsum(cost_mat_controlled, axis=1), axis=0), color = color_palette[1], alpha = 0.15)
    ax.fill_between(np.arange(cost_mat_v2g_home.shape[1]), np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0) - np.std(np.cumsum(cost_mat_v2g_home, axis=1), axis=0), np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0) + np.std(np.cumsum(cost_mat_v2g_home, axis=1), axis=0), color = color_palette[3], alpha = 0.15)
    ax.fill_between(np.arange(cost_mat_v2g_everywhere.shape[1]), np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0) - np.std(np.cumsum(cost_mat_v2g_everywhere, axis=1), axis=0), np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0) + np.std(np.cumsum(cost_mat_v2g_everywhere, axis=1), axis=0), color = color_palette[4], alpha = 0.15)

    #set legend
    ax.legend(loc='lower left', fontsize=13)

    #add light gray horizontal line at y=0
    ax.axhline(0, color='lightgray', linestyle='--', linewidth= 1)

    ax.set_xlabel('Month', fontsize=14)
    ax.set_ylabel('Cumul. Charging Cost [$1000]', fontsize=14)
    ax.set_xlim([0,num_periods+6])
    ax.set_xticks(np.linspace(0, num_periods-1, 5 ))
    ax.set_xticklabels(['Jan', 'April', 'July', 'Oct.', "Jan."], fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.annotate('$'+str(np.round(np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0)[-1], 0))[:-2], xy=(54.5, np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0)[-1]+60), xycoords= 'data', ha='center', fontsize=13)
    ax.annotate('$'+str(np.round(np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0)[-1], 0))[:-2], xy=(54.5, np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0)[-1]-60), xycoords= 'data', ha='center', fontsize=13)
    if np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1] < 0:
        ax.annotate('-$'+str(np.round(np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1], 0))[1:-2], xy=(54.5, np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1]+70), xycoords= 'data', ha='center', fontsize=13)
        ax.annotate('-$'+str(np.round(np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0)[-1], 0))[1:-2], xy=(54.5, np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0)[-1]-70), xycoords= 'data', ha='center', fontsize=13)
    else:
        ax.annotate('$'+str(np.round(np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1], 0))[:-2], xy=(54.5, np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1]-80), xycoords= 'data', ha='center', fontsize=13)
        ax.annotate('$'+str(np.round(np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0)[-1], 0))[:-2], xy=(54.5, np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0)[-1]+80), xycoords= 'data', ha='center', fontsize=13)

    min=-3500
    max=3000
    ax.set_ylim(min, max)
    ax.set_yticklabels([str(int(x/1000)) for x in np.arange(min-500, max+1, 1000)], fontsize=14)        
    ax.annotate('Circuit ' + str(circuit)[-1], xy=(0.1, .90), xycoords='axes fraction', ha='center', fontsize=16)


In [ ]:
def load_results(results_path, period_string):
    '''Load the cost results into matrices from the specified path and period string.'''
    vin_length = 749
    cost_mat_uncontrolled = []
    cost_mat_controlled = []
    cost_mat_v2g_home = []
    cost_mat_v2g_everywhere = []

    for vin in np.arange(1,vin_length):
        #skip the vins that do not have data 
        try:
            cost_mat_uncontrolled.append( pd.read_csv(results_path + 'cost_uncontrolled_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_controlled.append( pd.read_csv(results_path + 'cost_no_v2g_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_home.append( pd.read_csv(results_path + 'cost_v2g_home_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_everywhere.append(pd.read_csv(results_path + 'cost_v2g_everywhere_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])

        except:
            pass

    cost_mat_uncontrolled = np.array(cost_mat_uncontrolled)
    cost_mat_controlled = np.array(cost_mat_controlled)
    cost_mat_v2g_home = np.array(cost_mat_v2g_home)
    cost_mat_v2g_everywhere = np.array(cost_mat_v2g_everywhere)
    return cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere


In [ ]:
aging = 'batt_aging_0'
period_string = '2024-06-03_to_2025-06-01'

fig, ax = plt.subplots(2, 2, figsize=(15, 10))
ax = ax.flatten()
plt.subplots_adjust(wspace=0.18)
fig_label = ['a', 'b', 'c', 'd']

for i, circuit in enumerate(['cir_1', 'cir_2', 'cir_3', 'cir_4']):
    print('Circuit ' + str(circuit)[-1])
    #load results
    results_path = 'Results/'+ circuit+ '/' + aging + '/' + 'elrp_1_'
    cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere = load_results(results_path, period_string)
    cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere = replace_nan_with_previous_value(cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere)
    num_periods = cost_mat_uncontrolled.shape[1]
    plot_costs(ax[i], cost_mat_uncontrolled, cost_mat_controlled, cost_mat_v2g_home, cost_mat_v2g_everywhere, num_periods, circuit)
    print(str(np.round(np.mean(np.cumsum( cost_mat_uncontrolled, axis=1), axis=0)[-1],2)) + ' & ' + str(np.round(np.mean(np.cumsum( cost_mat_controlled, axis=1), axis=0)[-1], 2)) + ' & ' + str(np.round(np.mean(np.cumsum( cost_mat_v2g_home, axis=1), axis=0)[-1], 2)) + ' & ' + str(np.round(np.mean(np.cumsum( cost_mat_v2g_everywhere, axis=1), axis=0)[-1], 2)) )
    ax[i].annotate(fig_label[i]+'.', xy=(-0.1, 1.05), xycoords='axes fraction', ha='center', fontsize=16)

plt.savefig(save_path + '3_costs_no_clusters.pdf', bbox_inches='tight')